# **NVIDIA Riva RAG Chatbot Lab**

## Introduction

In this lab, you will build a **multilingual voice-enabled RAG chatbot** that combines NVIDIA Riva's speech AI capabilities with HPE AI Essentials Knowledge Base for intelligent, context-aware responses.

### What You'll Learn

- How to use **NVIDIA Riva** for multilingual speech recognition and synthesis
- How to integrate with **HPE AI Essentials Knowledge Base** for RAG-powered responses
- How to build an interactive voice interface for enterprise AI applications

### Technologies Used

**NVIDIA Riva:**
- **ASR (Automatic Speech Recognition)**: `nvidia/parakeet-1.1b-rnnt-multilingual-asr` - Converts speech to text with automatic language detection
- **TTS (Text-to-Speech)**: `nvidia/magpie-tts-multilingual` - Converts text responses back to natural speech

**HPE AI Essentials Knowledge Base:**
- Retrieval-Augmented Generation (RAG) framework for context-aware responses
- Vector database (Weaviate) for semantic search
- LLM: `meta/llama3-8b-instruct` for response generation

### Lab Flow

1. **Setup**: Install dependencies and configure endpoints
2. **Initialize Services**: Connect to Riva ASR and TTS services
3. **Define Functions**: Create ASR, RAG, and TTS functions
4. **Launch UI**: Start the interactive voice chatbot interface
5. **Interact**: Ask questions using voice, file upload, or saved recordings

---

## Architecture Overview

The following diagram illustrates the complete architecture of our voice-enabled RAG chatbot system:

![Riva RAG Architecture](./docs/Riva_RAG_Architecture.png)

### Component Breakdown

#### 1. **User Input** (Left - Teal Box)
The system accepts audio input through three methods:
- **🎤 Voice Input**: Real-time recording via browser microphone
- **📤 File Upload**: Upload pre-recorded audio files (.wav, .mp3)
- **📁 Saved Files**: Select from previously recorded/uploaded files in `./input/`

#### 2. **NVIDIA Riva ASR** (Top Center - Green Box)
Converts audio to text using the **Parakeet 1.1B Multilingual ASR** model:
- ✓ **Auto Language Detection**: Automatically identifies the spoken language
- ✓ **Multi-language Support**: English, Spanish, French, German, Chinese, and more
- ✓ **Automatic Punctuation**: Adds proper punctuation for readability

**Output**: Transcribed text sent to the RAG system

#### 3. **HPE AI Essentials Knowledge Base (RAG)** (Right - Teal Box)
The RAG pipeline processes queries through multiple stages:

**a. Vector Database (Weaviate)**
- Stores enterprise documents as high-dimensional vector embeddings
- Enables fast semantic similarity search

**b. Embedding Model (NVIDIA Retrieval QA E5 v5)**
- Converts text queries into numerical vectors
- Captures semantic meaning for accurate retrieval

**c. Semantic Search & Context Retrieval**
- Searches the vector database for relevant context
- Retrieves the most pertinent information for the query

**d. LLM Generation (Meta Llama3-8B-Instruct)**
- Generates responses based on retrieved context
- Follows system instructions to avoid hallucinations
- Produces responses in the selected output language

**Output**: Context-aware response text

#### 4. **Language Selection** (Left Center - Gray Box)
User-selectable output language:
- 🌍 English, Spanish, French, German, Chinese
- Sends **Output Language Code** to both RAG and TTS (dashed lines)
- RAG generates text in the selected language
- TTS synthesizes speech with appropriate accent

#### 5. **NVIDIA Riva TTS** (Bottom Center - Green Box)
Converts text responses to natural speech using **Magpie Multilingual TTS**:
- ✓ **Natural Voices**: Human-like speech synthesis
- ✓ **Multi-language**: Supports multiple languages with native accents
- ✓ **44.1kHz Audio**: High-quality audio output

**Output**: Audio response file

#### 6. **User Output** (Bottom Left - Teal Box)
The system presents results in multiple formats:
- **🔊 Audio Playback**: Auto-plays the synthesized speech
- **💾 Saved to ./outputs/**: Stores TTS audio files for later review
- **📝 Text Display**: Shows the transcribed query and RAG response

### Data Flow

```
1. User speaks or uploads audio → Audio File
2. NVIDIA Riva ASR processes → Transcribed Text
3. RAG system retrieves context → Context-Aware Response
4. NVIDIA Riva TTS synthesizes → Audio Response
5. User receives audio + text output
```

### Key Features

✅ **Multilingual Input**: Speak in any supported language (auto-detected)  
✅ **Multilingual Output**: Receive responses in your preferred language  
✅ **Context-Aware**: RAG retrieves relevant information from your knowledge base  
✅ **Voice-Enabled**: Complete hands-free interaction  
✅ **Persistent Storage**: All audio files saved for review  

---

## 1. Install Dependencies

Install the NVIDIA Riva client library and other required packages.

In [ ]:
%pip install nvidia-riva-client requests

---

## 2. Configuration

### Import Required Libraries

In [ ]:
import os
import wave
import json
import requests
import riva.client
import IPython.display as ipd
from IPython.display import display

### Configure Endpoints and Models

**Important:** Replace `YOUR_AUTH_TOKEN_HERE` and `APP_NAME_HERE` with your actual Knowledge Base authentication token and metadata name.

You can retrieve your token and metadata name from the HPE AI Essentials Knowledge Base settings page.

In [ ]:
# Riva Configuration
RIVA_URI = "10.179.253.43:32222"

# RAG Configuration
RAG_ENDPOINT = "https://rag-coordinator.pcai2.genai2.hou"
RAG_API_PATH = "/v1/chat/completions"
AUTH_TOKEN = "YOUR_AUTH_TOKEN_HER"  # Replace with your actual token
APP_NAME = "APP_NAME_HERE"  # Replace with your actual APP Name 
ENABLE_CITATIONS = "false"

# Models
ASR_MODEL = "nvidia/parakeet-1.1b-rnnt-multilingual-asr"
TTS_MODEL = "nvidia/magpie-tts-multilingual"
RAG_MODEL = "meta/llama3-8b-instruct"

print(f"Riva URI: {RIVA_URI}")
print(f"RAG Endpoint: {RAG_ENDPOINT}{RAG_API_PATH}")

### Define System Context

This system prompt instructs the RAG model to provide responses based strictly on retrieved context from the Knowledge Base.

In [ ]:
# System Context for RAG
SYSTEM_CONTEXT = """- You are an AI assistant who analyzes the input query and provides responses strictly based on the retrieved context and previous chat conversations.
- If no relevant context or prior chat conversations is available, do not respond and instead state: "I don't have enough information to answer the question."
- You must not rely on general knowledge or any data from your training. Use only the specific details from the given context and prior chat conversations.
- If no relevant context is retrieved, do not attempt to generate a response.
- Your answers must remain strictly within the boundaries of the available context and previous chat conversations.
- If the user sends a greeting or a polite remark, reply briefly in a friendly manner
- Do not include greetings in responses to non-greeting queries.
- Don't assume or hallucinate when responding
- Do not include the phrase 'Based on the provided context and previous chat conversation' in the responses.

Context:
{context} 

Chat_Conversations:
{chat_history}

Query:
{query}"""

---

## 3. Initialize Riva Services

Connect to the NVIDIA Riva server and initialize ASR and TTS services.

In [ ]:
try:
    auth = riva.client.Auth(uri=RIVA_URI)
    asr_service = riva.client.ASRService(auth)
    tts_service = riva.client.SpeechSynthesisService(auth)
    print("✓ Riva services initialized successfully.")
except Exception as e:
    print(f"✗ Error initializing Riva services: {e}")

---

## 4. Define ASR Function

This function transcribes audio files using the **Parakeet multilingual ASR model** with automatic language detection.

**Key Features:**
- Supports multiple languages (English, Spanish, French, German, Chinese, and more)
- Automatically detects the spoken language
- Enables automatic punctuation for better readability

In [ ]:
def transcribe_audio(audio_file):
    """Transcribe audio using Riva ASR with automatic language detection."""
    try:
        with open(audio_file, 'rb') as fh:
            data = fh.read()

        config = riva.client.RecognitionConfig(
            encoding=riva.client.AudioEncoding.LINEAR_PCM,
            max_alternatives=1,
            enable_automatic_punctuation=True,
            verbatim_transcripts=False,
            language_code="multi"  # Auto-detect language
        )
        
        # Helper to set sample_rate_hz and audio_channel_count from the WAV file header
        riva.client.add_audio_file_specs_to_config(config, audio_file)
        
        response = asr_service.offline_recognize(data, config)
        if len(response.results) > 0:
            transcript = response.results[0].alternatives[0].transcript
            return transcript
        else:
            return ""
    except Exception as e:
        print(f"ASR Error: {e}")
        return ""

---

## 5. Define RAG Query Function

This function sends the transcribed text to the **HPE AI Essentials Knowledge Base** RAG endpoint.

**How it works:**
1. Receives the user's transcribed query
2. Adds language instruction to the system context
3. Sends request to the RAG coordinator
4. Retrieves context from the vector database
5. Generates a response using the LLM

**Note:** The Knowledge Base handles context retrieval, embeddings, and chat history automatically.

In [ ]:
def query_rag(text, output_language="English"):
    """Query RAG endpoint with language-specific instruction."""
    url = f"{RAG_ENDPOINT}{RAG_API_PATH}"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {AUTH_TOKEN}",
        "X-SA-NAME": APP_NAME,
        "X-ENABLE-CITATIONS": ENABLE_CITATIONS
    }
    
    # Add language instruction to the system context
    language_instruction = f"\n\nIMPORTANT: Provide your response in {output_language} language."
    system_context_with_language = SYSTEM_CONTEXT + language_instruction
    
    # Payload for Chat Completion
    payload = {
        "model": RAG_MODEL,
        "messages": [
            {"role": "system", "content": system_context_with_language},
            {"role": "user", "content": text},
        ],
        "max_tokens": 2048,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 1,
        "top_p": 0.5
    }
    
    print(f"Querying Knowledge Base... (Requesting {output_language} response)")
    
    try:
        response = requests.post(url, headers=headers, json=payload, verify=False) 
        response.raise_for_status()
        result = response.json()
        
        # Handle Chat Completion Response
        if "choices" in result and len(result["choices"]) > 0:
            answer = result["choices"][0]["message"]["content"]
            return answer
        else:
            return f"Unexpected response format: {str(result)}"
            
    except Exception as e:
        print(f"RAG Error: {e}")
        return "I'm sorry, I couldn't connect to the knowledge base."

---

## 6. Define TTS Function

This function converts text responses to speech using the **Magpie multilingual TTS model**.

**Key Features:**
- Supports multiple languages with natural-sounding voices
- Saves audio files to `./outputs/` directory
- Auto-plays the generated audio in the notebook

In [ ]:
# Counter for unique filenames
tts_counter = 0
OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def speak_text(text, language_code="en-US"):
    """Synthesize speech in the specified language."""
    global tts_counter
    try:
        print(f"Synthesizing speech: {text[:50]}...")
        sample_rate_hz = 44100
        nchannels = 1
        sampwidth = 2
        
        response = tts_service.synthesize(
            text,
            language_code=language_code,
            sample_rate_hz=sample_rate_hz
        )
        
        # Save to output folder
        tts_counter += 1
        output_file = os.path.join(OUTPUT_DIR, f"response_{tts_counter}.wav")
        
        with wave.open(output_file, 'wb') as out_f:
            out_f.setnchannels(nchannels)
            out_f.setsampwidth(sampwidth)
            out_f.setframerate(sample_rate_hz)
            out_f.writeframesraw(response.audio)
        
        print(f"✓ Audio saved to: {output_file}")
        display(ipd.Audio(output_file, autoplay=True))
            
    except Exception as e:
        print(f"TTS Error: {e}")

---

## 7. Launch Interactive Chatbot UI

The chatbot UI provides three ways to interact:

### Input Options:
1. **Record Audio**: Click "Start Recording" to record your question, then "Stop Recording" when done
2. **Upload File**: Upload a pre-recorded audio file (.wav or .mp3)
3. **Select Saved File**: Choose from previously recorded or uploaded files in the `./input/` directory

### Output Language:
Select your preferred output language from the dropdown:
- English (US)
- Spanish (US)
- French
- German
- Chinese

### Complete Workflow:
```
Voice Input → ASR (auto-detect language) → RAG Query → LLM Response → TTS (selected language)
```

**Example:** Speak in English, get response generated and spoken in Spanish!

In [ ]:
from chatbot_ui import create_chatbot_ui

# Launch the chatbot UI
create_chatbot_ui(transcribe_audio, query_rag, speak_text)

---

## Summary

Congratulations! You've built a multilingual voice-enabled RAG chatbot that combines:

✅ **NVIDIA Riva ASR** - Multilingual speech recognition with auto-detection  
✅ **HPE AI Essentials Knowledge Base** - Context-aware RAG responses from your enterprise data  
✅ **NVIDIA Riva TTS** - Natural multilingual speech synthesis  

### Key Takeaways:
- Riva's Parakeet model automatically detects the spoken language
- Knowledge Base retrieves relevant context from your vector database
- Responses can be generated and spoken in any supported language
- All audio files are saved for future reference

### Next Steps:
- Explore the **Knowledge Base Lab** to learn how to build and manage your own RAG solutions
- Experiment with different languages and queries
- Review saved audio files in `./input/` and `./outputs/` directories